In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [22]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')

In [23]:
#question 1: Read the data for January. How many columns are there? 19
df.shape

(2463931, 19)

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [25]:
# Question 2: Now let's compute the duration variable. It should contain the duration of a ride in minutes.
# What's the standard deviation of the trips duration in January? 46.45
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)

In [26]:
df['duration'].std()

46.44530513776802

In [27]:
# Question 3
# Next, we need to check the distribution of the duration variable. 
# There are some outliers. 
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
# What fraction of the records left after you dropped the outliers? 98%

In [28]:
rows_before_drop_outliers = df.shape[0]
rows_before_drop_outliers

2463931

In [29]:
df = df[(df.duration >= 1) & (df.duration <=60)].copy()
rows_after_drop_outliers = df.shape[0]
rows_after_drop_outliers

2421440

In [30]:
rows_after_drop_outliers / rows_before_drop_outliers * 100

98.27547930522405

In [31]:
# Question 4
# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
# Turn the dataframe into a list of dictionaries
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)? 515

In [32]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str).copy()
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values
X_train.shape

(2421440, 515)

In [34]:
# Question 5
# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters
# Calculate the RMSE of the model on the training data
# What's the RMSE on train? 6.99

In [35]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190742248472

In [36]:
# Question 6
# Now let's apply this model to the validation dataset (February 2022).

# What's the RMSE on validation? 7.79
def read_data(filename:str) -> pd.DataFrame:
    df = pd.read_parquet(filename)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    
    df = df[(df.duration >= 1) & (df.duration <=60)].copy()
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str).copy()
    
    return df

In [37]:
df_val = read_data('data/yellow_tripdata_2022-02.parquet')

In [39]:
categorical = ['PULocationID', 'DOLocationID']
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.78640662117552